### Imports

In [1]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ealvarez/miniconda3/envs/Conda_dgl_3.10/li

time: 154 µs (started: 2023-06-10 14:54:50 -05:00)


### Parameters

In [2]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp1
Method:		 vgae
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 str
Subgroups id op: {'WT': ['str'], 'zwf1^': ['str'], 'pck1^': ['str']}
time: 3.73 ms (started: 2023-06-10 14:54:50 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  0.279032  1.974517  0.036078
  2  0.261945  3.289689  0.557086
  3  1.843187  2.303246  0.017578
  4  0.199392  3.910711  0.028566
  5  0.000288  6.111294  0.119179

time: 4.46 s (started: 2023-06-10 13:54:27 -05:00)


### Concat edge embeddings

In [ ]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [03:07<00:00, 62.47s/it]

time: 3min 7s (started: 2023-06-10 13:54:31 -05:00)


In [ ]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0         1         2  subgroup
0 1  0.279032  1.974517  0.036078         0
  2  0.261945  3.289689  0.557086         0
  3  1.843187  2.303246  0.017578         0
  4  0.199392  3.910711  0.028566         0
  5  0.000288  6.111294  0.119179         0

time: 4.47 s (started: 2023-06-10 13:57:39 -05:00)


In [ ]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 3/3 [14:00<00:00, 280.29s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 14min (started: 2023-06-10 13:57:43 -05:00)


### Outliers detection

In [ ]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 4.18 ms (started: 2023-06-10 14:11:44 -05:00)


In [ ]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [01:00<00:00, 20.13s/it]

time: 1min (started: 2023-06-10 14:11:44 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 1    0.279032  1.974517  0.036078         0
  2    0.261945  3.289689  0.557086         0
  8    0.844659  0.214240  0.160021         0
  9    0.523148  0.000579  0.062768         0
1 131  0.016571  0.177873  0.711692         0

time: 11 ms (started: 2023-06-10 14:12:45 -05:00)


In [ ]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 12257101
Num. of inliers: 11031391
Num. of inliers: 1225710


 33%|███▎      | 1/3 [04:46<09:33, 286.56s/it]

Total: 17101443
Num. of inliers: 15391298
Num. of inliers: 1710145


 67%|██████▋   | 2/3 [11:24<05:52, 352.31s/it]

...

100%|██████████| 3/3 [13:43<00:00, 274.55s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 13min 43s (started: 2023-06-10 14:12:45 -05:00)


###  Filter common edges

In [ ]:
print(subgroups_id)
print(subgroups_id_op)

{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['dyn'], 'zwf1^': ['dyn'], 'pck1^': ['dyn']}
time: 762 µs (started: 2023-06-10 14:26:29 -05:00)


In [ ]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11031391it [04:59, 36844.53it/s]it/s]
15391298it [06:55, 37030.70it/s]6, 308.03s/it]
5445463it [02:27, 37032.47it/s]17, 377.60s/it]
100%|██████████| 3/3 [14:45<00:00, 295.11s/it]

time: 14min 45s (started: 2023-06-10 14:26:29 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
A59.0049  A274.0123  0.279032  1.974517  0.036078         0
          A277.0867  0.261945  3.289689  0.557086         0
          A641.1223  0.844659  0.214240  0.160021         0
          A650.2145  0.523148  0.000579  0.062768         0
A274.0123 A191.046   0.016571  0.177873  0.711692         0

time: 12.8 ms (started: 2023-06-10 14:41:14 -05:00)


In [ ]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

11031391it [00:09, 1112178.03it/s]/s]
15391298it [00:14, 1075682.38it/s] 19.46s/it]
5445463it [00:04, 1210461.33it/s], 24.13s/it]
100%|██████████| 3/3 [00:56<00:00, 18.91s/it]

time: 56.7 s (started: 2023-06-10 14:41:15 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
59.0049  274.0123  0.279032  1.974517  0.036078         0
         277.0867  0.261945  3.289689  0.557086         0
         641.1223  0.844659  0.214240  0.160021         0
         650.2145  0.523148  0.000579  0.062768         0
274.0123 191.046   0.016571  0.177873  0.711692         0

time: 12.6 ms (started: 2023-06-10 14:42:11 -05:00)


In [ ]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:03<00:00,  1.08s/it]

time: 3.25 s (started: 2023-06-10 14:42:11 -05:00)


In [ ]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11008931, 4)


0         1         2  subgroup
59.0049  274.0123  0.279032  1.974517  0.036078         0
         277.0867  0.261945  3.289689  0.557086         0
         641.1223  0.844659  0.214240  0.160021         0
         650.2145  0.523148  0.000579  0.062768         0
274.0123 191.046   0.016571  0.177873  0.711692         0

time: 12.8 ms (started: 2023-06-10 14:42:15 -05:00)


In [ ]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # count
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["level_0", "level_1"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]

    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [00:13<00:00,  4.59s/it]

time: 13.8 s (started: 2023-06-10 14:42:15 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,714.5657,812.4817
1,146.0458,205.0829
2,283.2959,761.5157
3,716.5216,758.57
4,716.5216,758.5819


time: 9.08 ms (started: 2023-06-10 14:42:29 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26486 entries, 0 to 26485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  26486 non-null  object
 1   target  26486 non-null  object
dtypes: object(2)
memory usage: 620.8+ KB
time: 23.2 ms (started: 2023-06-10 14:42:29 -05:00)


In [ ]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 30.19it/s]

time: 104 ms (started: 2023-06-10 14:42:29 -05:00)


In [ ]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,714.5657,812.4817
1,146.0458,205.0829
2,283.2959,761.5157
3,716.5216,758.57
4,716.5216,758.5819


time: 6.07 ms (started: 2023-06-10 14:42:29 -05:00)


In [ ]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26486 entries, 0 to 26485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  26486 non-null  string
 1   target  26486 non-null  string
dtypes: string(2)
memory usage: 620.8 KB
time: 15.4 ms (started: 2023-06-10 14:42:29 -05:00)


In [ ]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [01:04<00:00, 21.46s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
15176,102.0562,96.9761,0.613406,0.573434,0.587553,0.830119,0.845229
21010,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
21006,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
20932,109.0407,144.0458,0.688714,0.710822,0.875062,0.922401,0.783513
21567,109.0407,272.0889,0.733025,0.594636,0.751374,0.822002,0.867299


time: 1min 4s (started: 2023-06-10 14:42:29 -05:00)


In [ ]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
15176,102.0562,96.9761,0.613406,0.573434,0.587553,0.830119,0.845229
21010,109.0407,127.0513,0.791174,0.792589,0.782065,0.917810,0.870420
21006,109.0407,131.0824,0.808212,0.663848,0.719276,0.857543,0.913011
20932,109.0407,144.0458,0.688714,0.710822,0.875062,0.922401,0.783513
21567,109.0407,272.0889,0.733025,0.594636,0.751374,0.822002,0.867299


time: 13.6 ms (started: 2023-06-10 14:43:34 -05:00)


### Filter by STD and average weight

In [ ]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

100%|██████████| 3/3 [00:12<00:00,  4.06s/it]


,source,target,weight
0,102.0562,96.9761,0.689948
1,109.0407,127.0513,0.830812
2,109.0407,131.0824,0.792378
3,109.0407,144.0458,0.796102
4,109.0407,272.0889,0.753667


<Figure size 640x480 with 0 Axes>

time: 12.2 s (started: 2023-06-10 14:43:34 -05:00)


In [ ]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,102.0562,96.9761,0.689948
1,109.0407,127.0513,0.830812
2,109.0407,131.0824,0.792378
3,109.0407,144.0458,0.796102
4,109.0407,272.0889,0.753667


time: 27 ms (started: 2023-06-10 14:43:46 -05:00)


In [ ]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

  0%|          | 0/3 [00:00<?, ?it/s]

Group: WT
Num. nodes: 2286
Num. edges: 26266



 67%|██████▋   | 2/3 [00:00<00:00,  2.03it/s]

Group: zwf1^
Num. nodes: 4883
Num. edges: 503193



100%|██████████| 3/3 [00:01<00:00,  1.69it/s]

Group: pck1^
Num. nodes: 5276
Num. edges: 438813

time: 1.78 s (started: 2023-06-10 14:43:46 -05:00)
